In [17]:
import nltk
from nltk.util import ngrams
import pandas as pd
import gensim
import contractions
import pickle

In [3]:
df = pd.read_csv('unique_data.csv')

In [4]:
df.head()


,review_stars,review_date,review_text,review_useful,review_funny,review_cool,user_name,user_review_count,user_yelping_since,user_useful,...,user_compliment_photos,tip_text,tip_date,tip_compliment_count,business_name,business_state,business_stars,business_review_count,business_categories,business_type
0,3.0,2014-09-10 23:19:21,Yet again.. Disappointed. Now they don't have ...,0,1,1,Alison,146,2011-12-05 22:07:12,220,...,0,Yum!!! Thanks for an amazing experience,2013-10-04 22:52:17,0,Keswick Tavern,PA,3.5,159,"Bars, Greek, Restaurants, Nightlife, Beer, Win...",F&B
1,4.0,2013-10-30 20:03:09,Ok really? When I see a bar that smells like ...,5,0,1,Alison,146,2011-12-05 22:07:12,220,...,0,Yum!!! Thanks for an amazing experience,2013-10-04 22:52:17,0,Keswick Tavern,PA,3.5,159,"Bars, Greek, Restaurants, Nightlife, Beer, Win...",F&B
2,3.0,2013-08-08 15:11:13,The K-T is a decent place. the bartenders are...,1,1,1,Alison,146,2011-12-05 22:07:12,220,...,0,Yum!!! Thanks for an amazing experience,2013-10-04 22:52:17,0,Keswick Tavern,PA,3.5,159,"Bars, Greek, Restaurants, Nightlife, Beer, Win...",F&B
3,3.0,2014-01-25 19:21:47,Food four stars!!!!! Bartender shorting me 10 ...,0,1,0,Alison,146,2011-12-05 22:07:12,220,...,0,Yum!!! Thanks for an amazing experience,2013-10-04 22:52:17,0,Keswick Tavern,PA,3.5,159,"Bars, Greek, Restaurants, Nightlife, Beer, Win...",F&B
4,4.0,2014-01-28 17:12:32,OK they get the star back for service. Even a...,1,1,1,Alison,146,2011-12-05 22:07:12,220,...,0,Yum!!! Thanks for an amazing experience,2013-10-04 22:52:17,0,Keswick Tavern,PA,3.5,159,"Bars, Greek, Restaurants, Nightlife, Beer, Win...",F&B


In [5]:
df.describe()

,review_stars,review_useful,review_funny,review_cool,user_review_count,user_useful,user_funny,user_cool,user_fans,user_average_stars,...,user_compliment_list,user_compliment_note,user_compliment_plain,user_compliment_cool,user_compliment_funny,user_compliment_writer,user_compliment_photos,tip_compliment_count,business_stars,business_review_count
count,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,...,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000,102326.000000
mean,4.064402,3.220423,1.207718,2.108604,482.571321,2979.490364,1319.419522,2210.992065,94.561930,3.947441,...,3.891113,136.283711,399.515861,263.000303,263.000303,85.245509,128.731808,0.023015,3.792057,391.609454
std,1.013901,6.224662,3.455563,5.449386,631.814355,8296.960979,4819.579650,6965.832324,304.943573,0.326913,...,26.467907,486.853147,1464.561770,896.127814,896.127814,263.318674,689.151990,0.166504,0.619643,697.951534
min,1.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,0.000000,2.440000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000
25%,4.000000,0.000000,0.000000,0.000000,140.000000,225.000000,50.000000,96.000000,10.000000,3.730000,...,0.000000,8.000000,9.000000,10.000000,10.000000,6.000000,2.000000,0.000000,3.500000,67.000000
50%,4.000000,2.000000,0.000000,1.000000,273.000000,562.000000,151.000000,271.000000,24.000000,3.960000,...,0.000000,20.000000,26.000000,31.000000,31.000000,17.000000,7.000000,0.000000,4.000000,179.000000
75%,5.000000,4.000000,1.000000,2.000000,552.000000,1767.000000,592.000000,1009.000000,65.000000,4.170000,...,1.000000,66.000000,118.000000,122.000000,122.000000,54.000000,39.000000,0.000000,4.000000,418.000000
max,5.000000,360.000000,141.000000,360.000000,17473.000000,206296.000000,185823.000000,195814.000000,12497.000000,5.000000,...,2413.000000,59031.000000,101097.000000,49967.000000,49967.000000,15934.000000,56104.000000,5.000000,5.000000,7568.000000


In [6]:
# Label the data with new column where review_stars is greater than 3 is positive and less than 3 is negative and equal to 3 is neutral
df['sentiment'] = df['review_stars'].apply(lambda x: 'positive' if x > 3 else 'negative' if x < 3 else 'neutral')

In [8]:
# Preprocess the data
stop_list = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

# We use the following list to store the sequence of sentence labels.
labels = []

# We use the following list to store the sentences, where each sentence itself is a list of words.
corpus = []

# For every row in the data frame
for index, row in df.iterrows():
    # Extract the label and the text.
    label = row['sentiment']
    text = row['review_text']
    
    # Store the label into the list of labels.
    labels.append(label)
    
    # Tokenize the text.
    sent = nltk.word_tokenize(text)
    
    # Lowercase conversion
    sent = [w.lower() for w in sent]
    
    # Stop word removal 
    sent = [w for w in sent if w not in stop_list]

    # Remove punctuation
    sent = [w for w in sent if w.isalnum()]
    
    # Lemmatization 
    sent = [lemmatizer.lemmatize(w) for w in sent]

    # Expand contractions
    sent = [contractions.fix(w) for w in sent]

    # Create bigrams
    bigrams = [' '.join(w) for w in list(ngrams(sent, 2))]
    sent.extend(bigrams)
    
    
    # Store the sentence into the corpus.
    corpus.append(sent)

print('Finished reading sentences.')


Finished reading sentences.


In [9]:
# Create a dictionary from the corpus.
dictionary = gensim.corpora.Dictionary(corpus)

# Store the labeled training data in the following list.
labeled_data = []
count = 0
# Going through the two lists in parallel to create the labeled data set.
for (l, s) in zip(labels, corpus):

    # Convert the original sentence into a vector.
    vector = dictionary.doc2bow(s)
    
    # Create a dict object to store the document vector (in order to use NLTK's classifier later)
    sent_as_dict = {id:1 for (id, tf) in vector}
    
    # Add the labeled sentence to the labeled data set.
    labeled_data.append((sent_as_dict, l))
    
print('Finished preparing the data.')

Finished preparing the data.


In [18]:
with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(dictionary, handle)


In [10]:
# Split the data into training, validation, and test sets using 60%, 20%, 20% respectively.
train_size = int(len(labeled_data) * 0.6)
val_size = int(len(labeled_data) * 0.2)
train_data = labeled_data[:train_size]
val_data = labeled_data[train_size:train_size+val_size]
test_data = labeled_data[train_size+val_size:]

print('Finished splitting the data.')



Finished splitting the data.


In [11]:
numIterations = 10
algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
entclassifier = nltk.MaxentClassifier.train(train_data, algorithm, max_iter=numIterations)

  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.791
             2          -1.09855        0.791
             3          -1.09848        0.791
             4          -1.09841        0.791
             5          -1.09835        0.791
             6          -1.09828        0.791
             7          -1.09822        0.791
             8          -1.09815        0.791
             9          -1.09808        0.791
         Final          -1.09802        0.791


In [15]:
# Evaluate base on accuracy and f1_score
import collections
from nltk.metrics import f_measure

# Compute f1 scores for each label.
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, (feats, label) in enumerate(test_data):
    refsets[label].add(i)
    observed = entclassifier.classify(feats)
    testsets[observed].add(i)

accuracy = nltk.classify.accuracy(entclassifier, test_data)
print( 'F1 score:', f_measure(refsets['positive'], testsets['positive']) )
print('Validation accuracy:', accuracy)


F1 score: 0.8562327102020285
Validation accuracy: 0.7486074464966286


In [16]:
# Save the model
import pickle

with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(entclassifier, f)

    

In [21]:
# Predict the sentiment of a new review
def predict_sentiment(text):
    sent = nltk.word_tokenize(text)
    
    # Lowercase conversion
    sent = [w.lower() for w in sent]
    
    # Stop word removal 
    sent = [w for w in sent if w not in stop_list]

    # Remove punctuation
    sent = [w for w in sent if w.isalnum()]
    
    # Lemmatization 
    sent = [lemmatizer.lemmatize(w) for w in sent]

    # Expand contractions
    sent = [contractions.fix(w) for w in sent]

    # Create bigrams
    bigrams = [' '.join(w) for w in list(ngrams(sent, 2))]
    
    sent.extend(bigrams)
    
    # Convert the original sentence into a vector.
    vector = dictionary.doc2bow(sent)
    
    # Create a dict object to store the document vector (in order to use NLTK's classifier later)
    sent_as_dict = {id:1 for (id, tf) in vector}
    
    # Predict the sentiment of the review.
    return entclassifier.classify(sent_as_dict)

In [25]:
# Predict text
text = "BEST experience I have ever had at the KT on 6/14/2015.  John, our server, was amazing.  Really awesome.  The food was awesome as well.  I was sooooo happy we went.  Great job guys!!  Keep up the great work.  BTW the burgers are amazing.  Thanks again!"
print(predict_sentiment(text))

positive
